#### 1. Organizar o código da aula 04 em funções.

In [78]:
import pandas as pd
from geopy.geocoders import Nominatim

def show_types(data):
    print(data.dtypes)
    return None


def show_dimensions(data):
    print(f'\nNumber of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}', end='\n\n')
    return None


def data_collect(path):
    data = pd.read_csv(path)
    
    show_dimensions(data)
    
    show_types(data)
    
    return data


def collect_geodata(data, cols):
    geolocator = Nominatim(user_agent='geoapiExercises')
    
    data = data.head(20)
    
    data.loc[:, cols] = 'NA'
    
    for i in range(len(data)):
        if (i+1) % 10 == 0:
            print(f'Loop: {i+1}/{len(data)}')
        
        query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
        response = geolocator.reverse(query)   
        
        for c in range(len(cols)):
            if cols[c] in response.raw['address']:    
                data.loc[i, cols[c]] = response.raw['address'][cols[c]]        
            
    return data
        

def data_transform(data):
    data['level'] = data['price'].apply(lambda x: 0 if x < 321950 else
                                          1 if (x >= 321950) & (x < 450000) else
                                          2 if (x >= 450000) & (x < 645000) else 3)
    
    show_dimensions(data)
    show_types(data)
    
    cols = ['road', 'house_number', 'neighbourhood', 'city', 'county', 'state']
    df =  data.head(20)    
    df1 = collect_geodata(df, cols)
    show_dimensions(df1)
    print(df1.head())
    
    return data
    

if __name__ == '__main__':
    data_raw = data_collect('datasets/kc_house_data.csv')

    data_processing = data_transform(data_raw)

    print(data_processing)


Number of rows: 21613 
Number of columns: 21

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

Number of rows: 21613 
Number of columns: 22

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built         

C:\Users\henri\AppData\Local\Temp\ipykernel_2820\2976063689.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, cols] = 'NA'


Loop: 10/20
Loop: 20/20

Number of rows: 20 
Number of columns: 28

           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...     long  sqft_living15  \
0      5650     1.0           0     0  ... -122.257           1340   
1      7242     2.0           0     0  ... -122.319           1690   
2     10000     1.0           0     0  ... -122.233           2720   
3      5000     1.0           0     0  ... -122.393           1360   
4      8080     1.0           0     0  ... -122.045           1800   

   sqft_lot15  level                      

Number of rows: 21613 
Number of columns: 21
